# 130. Item-Based Collaborative Filtering (아이템 기반 협업 필터링)

### User-Based CF vs Item_Basef CF

| 사용자 | 기생충 | 겨울왕국 | 부산행 | 백두산 | 
| --- | --- |--- |--- | ---|
| 철수|4 | 3 | 5 |   |  
|영희| |2|1|2|
|길동|1|5| | 3| 
|정숙| | |4|5|

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FpxM35%2FbtrbAIgrefV%2FKPUOwAAuXRUw5DOaQshNi1%2Fimg.png" width="500" />

### UBCF

- 장점 : 각 사용자별로 맞춤형 추천을 하므로 정확한 추천이 가능
- 단점 : 
    - 데이터가 풍부한 경우(구매나 평가 정보) 추천이 가능   
    - 데이터가 조금만 바뀌어도 업데이트 필요
    

### IBCF

- 장점 :
    - 사용자별로 따로 계산 않으므로 계산 속도가 빠르다.
    - 데이터가 조금 바뀌어도 추천 결과에 영향이 크지 않으므로 업데이트를 자주하지 않아도 된다.  
    - 대규모 사이트에 적합
- 단점 : 사용자의 최신 취향 반영에 시간이 걸릴 수 있음


In [1]:
import sklearn
sklearn.__version__

'1.2.2'

In [3]:
# 필요한 라이브러리 임포트
import numpy as np  
import pandas as pd  
from sklearn.model_selection import train_test_split  

# from sklearn.metrics import root_mean_squared_error   # sklearn 최신 버전
from sklearn.metrics import mean_squared_error  # sklearn 구 버전

# 두 벡터 간의 코사인 유사도를 계산하기 위한 함수
from sklearn.metrics.pairwise import cosine_similarity

# 영화 데이터의 열 이름을 정의
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']

# 영화 (특성) 정보 로드
movies = pd.read_csv('data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head(3)

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


- content-based filtering 과 달리 CF 는 similarity 를 이용하므로 **item 정보가 필요하지 않기 때문에** movies dataframe에서 title 을 제외한 모든 다른 열을 drop 한다.  

In [4]:
# item 정보가 필요하지 않으므로 movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]
movies.head(3)

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)


In [5]:
# 평점 데이터의 열 이름을 정의
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

# 평점 데이터를 파일에서 읽어옴
# 'data/u.rating' 파일을 탭('\t')으로 구분된 데이터로 읽고, 열 이름을 r_cols로 지정
ratings = pd.read_csv('data/u.rating', sep='\t', names=r_cols, encoding='latin-1')
ratings

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


- ratings 만을 입력으로 사용

In [6]:
# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
ratings.head(3)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1


### 평점 Matrix 생성 

- 아이템 간의 유사도 계산은 아이템 자체의 특성을 비교하는 것이므로, ratings를 UF 처럼 훈련과 테스트 데이터로 분리할 필요가 없습니다

In [7]:
# movie_id를 인덱스로, user_id를 컬럼으로 사용하여
# 각 사용자가 각 영화에 부여한 평점을 값으로 하는 사용자-영화 평점 매트릭스를 생성합니다.
rating_matrix = ratings.pivot(index="movie_id", columns="user_id", values="rating")

print(rating_matrix.shape)
rating_matrix

(1682, 943)


user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Item-Based CF 알고리즘

1) 모든 Item 간의 평가의 유사도를 계산  
2) 현재 추천 대상이 되는 아이템과 다른 아이템들의 유사도를 추출  
3) 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가 값 계산. 예상 평가 값은 추천하려는 영화와 사용자가 평가한 영화의 유사도로 가중 평균  
4) 아이템 중에서 예상 평가 값이 가장 높은 N 개의 아이템을 추천  

이때, 추천 대상 영화와 가장 유사도가 높은 K 개의 다른 영화만 반영하여 예측 평가의 품질을 높인다.

이를 위해 추천 사용자, 영화 id 를 parameter 로 받아 예상 평점을 반환하는 함수를 작성

In [8]:
# 평점 매트릭스의 복사본을 생성하고, NaN(평점을 부여하지 않은 영화) 값을 0으로 채웁니다.
matrix_dummy = rating_matrix.copy().fillna(0)
print(matrix_dummy.shape)

# 모든 가능한 아이템(movie) pair의 Cosine similarities 계산
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix.index, columns=rating_matrix.index)

print(item_similarity.shape)
item_similarity.head(10)

(1682, 943)
(1682, 1682)


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.402382,0.330245,0.454938,0.286714,0.116344,0.620979,0.481114,0.496288,0.273935,...,0.035387,0.000000,0.000000,0.000000,0.035387,0.0,0.0,0.0,0.047183,0.047183
2,0.402382,1.000000,0.273069,0.502571,0.318836,0.083563,0.383403,0.337002,0.255252,0.171082,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.078299,0.078299
3,0.330245,0.273069,1.000000,0.324866,0.212957,0.106722,0.372921,0.200794,0.273669,0.158104,...,0.000000,0.000000,0.000000,0.000000,0.032292,0.0,0.0,0.0,0.000000,0.096875
4,0.454938,0.502571,0.324866,1.000000,0.334239,0.090308,0.489283,0.490236,0.419044,0.252561,...,0.000000,0.000000,0.094022,0.094022,0.037609,0.0,0.0,0.0,0.056413,0.075218
5,0.286714,0.318836,0.212957,0.334239,1.000000,0.037299,0.334769,0.259161,0.272448,0.055453,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.094211
6,0.116344,0.083563,0.106722,0.090308,0.037299,1.000000,0.139617,0.083876,0.151064,0.203097,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
7,0.620979,0.383403,0.372921,0.489283,0.334769,0.139617,1.000000,0.423515,0.527462,0.318623,...,0.000000,0.051498,0.000000,0.000000,0.051498,0.0,0.0,0.0,0.051498,0.051498
8,0.481114,0.337002,0.200794,0.490236,0.259161,0.083876,0.423515,1.000000,0.424429,0.267764,...,0.000000,0.082033,0.065627,0.065627,0.082033,0.0,0.0,0.0,0.082033,0.000000
9,0.496288,0.255252,0.273669,0.419044,0.272448,0.151064,0.527462,0.424429,1.000000,0.288514,...,0.000000,0.000000,0.057360,0.057360,0.071700,0.0,0.0,0.0,0.057360,0.071700


현재 추천 대상이 되는 영화와 다른 영화의 유사도를 추출

In [9]:
user_id = 2   # 현재 추천 대상이 되는 사용자
movie_id = 2  # 현재 추천 대상이 되는 영화

# 추천 대상 영화와 다른 영화 간의 유사도 가져오기
sim_scores = item_similarity.loc[movie_id]

print(sim_scores.shape)
sim_scores.head()

(1682,)


movie_id
1    0.402382
2    1.000000
3    0.273069
4    0.502571
5    0.318836
Name: 2, dtype: float64

### 추천 대상이 되는 사용자에게 보지 않은 영화를 추천할 때 사용자가 부여할 rating 예측

1. 현 사용자의 모든 rating값 가져오기  
2. 현 사용자가 평가하지 않은 영화의 index 가져오기
3. 현 사용자가 평가하지 않은 영화의 rating (null) 제거 --> 사용자가 평가한 영화만 남김

In [10]:
# 1. rating maxtrix에서 추천 대상 사용자의 모든 영화 평가 가져오기
user_ratings = rating_matrix.loc[:, user_id]

print(user_ratings.shape)
user_ratings.head()

(1682,)


movie_id
1    4.0
2    NaN
3    NaN
4    NaN
5    NaN
Name: 2, dtype: float64

In [11]:
# 2. 추천 대상 사용자가 평가하지 않은 영화의 인덱스 가져오기
non_rating_idx = user_ratings[user_ratings.isnull()].index

print(non_rating_idx.shape)
non_rating_idx

(1620,)


Index([   2,    3,    4,    5,    6,    7,    8,    9,   11,   12,
       ...
       1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682],
      dtype='int64', name='movie_id', length=1620)

In [12]:
# 3. 추천 대상 사용자가 평가하지 않은 영화의 평점 제거 --> 사용자가 평가한 영화만 남김
user_ratings = user_ratings.drop(non_rating_idx)

print(user_ratings.shape)
user_ratings.head()

(62,)


movie_id
1     4.0
10    2.0
13    4.0
14    4.0
19    3.0
Name: 2, dtype: float64

In [13]:
# item 유사도 matrix에서 추천 대상 사용자가 평가하지 않은 영화의 유사도 제거
# 추천 대상 영화와 다른 영화 간의 유사도 중 사용자가 평가한 영화만 남김
sim_scores = sim_scores.drop(non_rating_idx)

sim_scores /= sim_scores.sum()  # 유사도 정규화
print(sim_scores.shape)
sim_scores

(62,)


movie_id
1      0.034677
10     0.014744
13     0.018440
14     0.010812
19     0.002645
         ...   
312    0.005714
313    0.021919
314    0.003018
315    0.015032
316    0.017065
Name: 2, Length: 62, dtype: float64

### 추천하려는 영화에 대한 추천 대상 사용자의 예상 rating 계산

In [15]:
n_neighbors = 10
# 4. 추천 대상 영화와 유사도가 높은 영화 순으로 정렬 하여 가장 유사도 높은 n 개 선택
top_items = sim_scores.sort_values(ascending=False).head(n_neighbors)
top_items

movie_id
50     0.036788
1      0.034677
127    0.030979
100    0.030208
273    0.029938
257    0.027601
25     0.026426
281    0.026376
284    0.026244
282    0.025840
Name: 2, dtype: float64

In [16]:
# 5. top_items.index에 해당하는 영화의 평점을 user_ratings에서 재정렬하여 추출합니다.
# 이는 가장 유사도가 높은 상위 아이템의 평점을 선택하는 작업입니다.
# user_ratings 데이터프레임을 top_items의 인덱스 순서대로 재정렬
top_ratings = user_ratings.reindex(top_items.index) 
# NaN 값을 가진 요소들을 제거합니다.
top_ratings = top_ratings.dropna()
top_ratings

movie_id
50     5.0
1      4.0
127    5.0
100    5.0
273    4.0
257    4.0
25     4.0
281    3.0
284    4.0
282    4.0
Name: 2, dtype: float64

In [17]:
# KNN을 이용한 추천 공식 계산을 위해 유효한 top_items와 top_ratings만 사용합니다.
valid_indices = top_ratings.index  
top_items = top_items.loc[valid_indices]
top_items

movie_id
50     0.036788
1      0.034677
127    0.030979
100    0.030208
273    0.029938
257    0.027601
25     0.026426
281    0.026376
284    0.026244
282    0.025840
Name: 2, dtype: float64

### Implicit Rating을 이용한 추천 공식

$$\tilde{r}_{ui} = \frac{\sum_{j \in KNN(i)}r_{uj} \cdot S_{ji}}{\sum_{j \in KNN(i)}S_{ji}}$$

In [18]:
# 6. KNN 평점 예측 계산
# 추천대상 사용자 u에게 영화 i를 추천 했을 때 예상 rating
predicted_rating = np.dot(top_items, top_ratings) / top_items.sum()
predicted_rating

4.242645126277608

###  특정 사용자에게 추천하는 영화(movie_id)의  가중평균 rating을 계산하는 함수
- 위의 과정을 model 함수로 작성
- 위에서 개별 사용자와 아이템에 대해 구하던 predicted_rating을 함수로 작성
- 가중치는 주어진 영화와 주어진 사용자가 평가한 영화의 유사도(user_similarity)

In [19]:
def IBCF_knn_model(user_id, movie_id, rating_matrix, item_similarity, n_neighbors=0):
    """
    Item-Based Collaborative Filtering을 사용하여 특정 영화에 대한 사용자의 예상 평점을 계산합니다.

    Parameters:
    - user_id (int): 평점을 예측하고자 하는 사용자의 ID.
    - movie_id (int): 예상 평점을 계산하고자 하는 영화의 ID.
    - rating_matrix (DataFrame): 사용자-영화 평점 행렬. 사용자 ID가 인덱스이고, 영화 ID가 컬럼입니다.
    - item_similarity (DataFrame): 영화 간의 유사도를 나타내는 데이터프레임. 영화 ID가 인덱스이고 컬럼입니다.
    - n_neighbors (int): 고려할 이웃의 수. 0인 경우 모든 이웃을 고려합니다. 기본값은 0입니다.

    Returns:
    - predicted_rating (float): 지정된 영화에 대한 사용자의 예상 평점. 예상 평점을 계산할 수 없는 경우 기본값으로 3.0을 반환합니다.

    이 함수는 사용자가 평가한 영화의 평점과 해당 영화와 유사한 영화 간의 유사도를 기반으로 예상 평점을 계산합니다.
    유사도와 평점 데이터를 사용하여 가중평균을 계산하며, 특정 조건에서는 기본 평점을 할당합니다.
    """
    # 현재 영화가 item_similarity 인덱스에 있는지 확인
    if movie_id in item_similarity.index:  
        # 현재 영화와 다른 영화 간의 유사도 가져오기
        sim_scores = item_similarity.loc[movie_id]
        # 1. 현재 사용자의 모든 평가 가져오기
        user_ratings = rating_matrix.loc[user_id]
        # 2. 사용자가 평가하지 않은 영화의 인덱스 가져오기
        non_rating_idx = user_ratings[user_ratings.isnull()].index
        # 3. 사용자가 평가하지 않은 영화의 평점과 유사도 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        user_ratings = user_ratings.drop(non_rating_idx)
        
        # 유사도의 합이 0이 아닌 경우에만 계산
        if sim_scores.sum() > 0:
            # 유사도 정규화
            sim_scores /= sim_scores.sum()
            
            if n_neighbors == 0:  # n_neighbors가 지정되지 않은 경우 모든 유사도 사용
                n_neighbors = len(sim_scores)
            
            # n_neighbors가 1 이상이면, 유사도가 높은 상위 n_neighbors만 고려
            if n_neighbors > 1 and len(sim_scores) > 1:
                # 4. 유사도가 높은 아이템 순으로 정렬
                top_items = sim_scores.sort_values(ascending=False).head(n_neighbors)
                # 5. 상위 n_neighbors에 해당하는 valid한 평점만 추출
                # reindex를 사용하여 해당 인덱스가 없는 경우 NaN 값으로 채워 넣습니다.
                top_ratings = user_ratings.reindex(top_items.index)
                # NaN 값을 가진 요소들을 제거합니다.
                top_ratings = top_ratings.dropna()
                # # KNN을 이용한 추천 공식 계산을 위해 유효한 top_items와 top_ratings만 사용합니다.
                valid_indices = top_ratings.index  
                top_items = top_items.loc[valid_indices]
                # 6. KNN 평점 예측 계산
                if top_items.sum() > 0:
                    predicted_rating = np.dot(top_items, top_ratings) / top_items.sum()
                else:
                    predicted_rating = 3.0
            else:
                predicted_rating = 3.0  # 유사한 아이템이 충분하지 않은 경우 기본 평점 할당
        else:
            predicted_rating = 3.0  # 유사도의 합이 0인 경우 기본 평점 할당
    else:
        predicted_rating = 3.0  # 영화가 item_similarity에 없으면 기본 평점 할당
    return predicted_rating

In [20]:
IBCF_knn_model(12, 203, rating_matrix, item_similarity, n_neighbors=30)

3.9124701836089573

In [21]:
IBCF_knn_model(255, 51, rating_matrix, item_similarity, n_neighbors=30)

3.391002868026238

In [22]:
IBCF_knn_model(13, 9999, rating_matrix, item_similarity, n_neighbors=30)

3.0

### 모델 성능 평가 - 모델별 RMSE를 계산하는 함수 
- Test set을 이용하여 측정  

In [23]:
X = ratings.copy()

_, X_test = train_test_split(X, test_size=0.25, random_state=0)
X_test.shape

(25000, 3)

In [24]:
def score(model, n_neighbors=0):
    # X_test 데이터셋에서 사용자 ID와 영화 ID의 쌍을 생성합니다.
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    
    # 생성된 각 ID 쌍에 대해 모델을 사용하여 예상 평점을 계산합니다.
    y_pred = np.array([model(user, movie, rating_matrix, item_similarity, n_neighbors) for (user, movie) in id_pairs])
    
    # X_test 데이터셋에서 실제 평점을 가져옵니다.
    y_true = np.array(X_test['rating'])
    
    # 실제 평점과 예측 평점 사이의 RMSE를 계산하여 반환합니다.
    #return root_mean_squared_error(y_true, y_pred)  # sklearn latest version
    return mean_squared_error(y_true, y_pred, squared=False)  # sklearn old version

In [25]:
%%time
# 정확도 계산
print("knn 감안하지 않은 simple IF - 모든 item 유사도를 포함")
print(score(IBCF_knn_model, n_neighbors=0))

K = 30
print(f"knn 감안한 IF - {K} 개의 가장 유사한 item만 포함")
print(score(IBCF_knn_model, n_neighbors=K))

knn 감안하지 않은 simple IF - 모든 item 유사도를 포함
1.331414149683147
knn 감안한 IF - 30 개의 가장 유사한 item만 포함
1.362714869794291
CPU times: total: 18 s
Wall time: 44.6 s


### 특정 사용자에 대하여 영화 추천

In [29]:
# 추천 대상 사용자가 평가한 영화 평점 가져오기
# 추천 대상 사용자의 ID 설정
user_id = 2  
# rating_matrix에서 해당 사용자가 평가한 영화 평점 가져오기
user_movie_ratings = rating_matrix.loc[:, user_id]  

# 추천 대상 사용자가 평가한 영화 평점 출력
user_movie_ratings

movie_id
1       4.0
2       NaN
3       NaN
4       NaN
5       NaN
       ... 
1678    NaN
1679    NaN
1680    NaN
1681    NaN
1682    NaN
Name: 2, Length: 1682, dtype: float64

In [30]:
# 추천 대상 사용자에 의해 평가되지 않은 영화만 필터링 
unrated_movies = user_movie_ratings[user_movie_ratings.isnull()]
unrated_movies.index[:20]

Index([2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 15, 16, 17, 18, 20, 21, 22, 23, 24, 26], dtype='int64', name='movie_id')

In [31]:
# 사용자가 평가하지 않은 각 영화에 대해 IBCF 방식으로 예상 평점을 계산
predicted_ratings = {movie: IBCF_knn_model(user_id, movie, rating_matrix, item_similarity, n_neighbors=30)
                     for movie in unrated_movies.index}

# 예상 평점을 포함하는 Series 객체 생성
predicted_ratings = pd.Series(predicted_ratings)
predicted_ratings.head()

2    3.200713
3    3.225232
4    3.043486
5    2.974994
6    3.438292
dtype: float64

영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌

In [32]:
# 영화를 예상 평점에 따라 정렬하고, 상위 10개의 영화를 추천 목록으로 선정
top_movies = predicted_ratings.sort_values(ascending=False).head(10)
top_movies

1086    5.000000
1405    5.000000
695     5.000000
1604    5.000000
1324    5.000000
1434    5.000000
1278    5.000000
1592    4.569907
700     4.525458
1171    4.463796
dtype: float64

In [33]:
# 추천 영화의 제목 반환
recommendations = movies.loc[top_movies.index]['title']
recommendations

1086                                  Bloodsport 2 (1995)
1405                         When Night Is Falling (1995)
695                                      City Hall (1996)
1604                                 Love Serenade (1996)
1324                                        August (1996)
1434                       Steal Big, Steal Little (1995)
1278                                  Wild America (1997)
1592                            Death in Brunswick (1991)
700     Wonderful, Horrible Life of Leni Riefenstahl, ...
1171                                    Women, The (1939)
Name: title, dtype: object

### 특정 사용자에 대하여 추천 받는 함수

In [34]:
def recom_movie(user_id, n_items, neighbor_size=30):
    # 현 사용자가 평가한 영화 ratings 가져오기
    user_movie_ratings = rating_matrix.loc[:, user_id]
    
    # 평가되지 않은 영화만 필터링
    unrated_movies = user_movie_ratings[user_movie_ratings.isnull()]
    
    # 사용자가 평가하지 않은 각 영화에 대해 IBCF 방식으로 예상 평점을 계산
    predicted_ratings = {movie: IBCF_knn_model(user_id, movie, rating_matrix, item_similarity, neighbor_size)
                         for movie in unrated_movies.index}
    
    # 예상 평점을 포함하는 Series 객체 생성
    predicted_ratings = pd.Series(predicted_ratings)
    
    # 영화를 예상 평점에 따라 정렬하고, 상위 n_items 개의 영화를 추천 목록으로 선정
    top_movies = predicted_ratings.sort_values(ascending=False).head(n_items)
    
    # 추천 영화의 제목 반환
    recommendations = movies.loc[top_movies.index]['title']
    return recommendations

In [35]:
recom_movie(user_id=2, n_items=5, neighbor_size=30)

1086             Bloodsport 2 (1995)
1405    When Night Is Falling (1995)
695                 City Hall (1996)
1604            Love Serenade (1996)
1324                   August (1996)
Name: title, dtype: object

### 최적의 neighbor size 구하기

In [36]:
%%time
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print(f"Neighbor size = {neighbor_size} : RMSE = {score(IBCF_knn_model, neighbor_size):.4f}")

Neighbor size = 10 : RMSE = 1.3954
Neighbor size = 20 : RMSE = 1.3698
Neighbor size = 30 : RMSE = 1.3627
Neighbor size = 40 : RMSE = 1.3603
Neighbor size = 50 : RMSE = 1.3569
Neighbor size = 60 : RMSE = 1.3564
CPU times: total: 53.2 s
Wall time: 2min 32s
